<a href="https://colab.research.google.com/github/lqh52/Early-prediction-of-sepsis/blob/main/Add_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_pickle('/content/drive/MyDrive/Sepsis_predict/combined.pkl')
df

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,FiO2,pH,PaCO2,SaO2,AST,BUN,Alkalinephos,Calcium,Chloride,Creatinine,Bilirubin_direct,Glucose,Lactate,Magnesium,Phosphate,Potassium,Bilirubin_total,TroponinI,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel,patient
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,1,0,000001
1,97.0,95.0,NaN,98.0,75.33,NaN,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,2,0,000001
2,89.0,99.0,NaN,122.0,86.00,NaN,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,3,0,000001
3,90.0,95.0,NaN,NaN,NaN,NaN,30.0,NaN,24.0,NaN,NaN,7.36,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,4,0,000001
4,103.0,88.5,NaN,122.0,91.33,NaN,24.5,NaN,NaN,NaN,0.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.14,0,NaN,NaN,-0.03,5,0,000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1552205,76.0,99.0,37.3,137.0,98.00,79.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,164.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.00,0,0.0,1.0,-13.25,33,0,109334
1552206,90.0,100.0,NaN,116.0,88.00,70.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.00,0,0.0,1.0,-13.25,34,0,109334
1552207,90.0,100.0,NaN,118.0,84.00,64.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.00,0,0.0,1.0,-13.25,35,0,109334
1552208,82.0,100.0,NaN,114.0,83.00,66.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.00,0,0.0,1.0,-13.25,36,0,109334


In [ ]:
#Quick SOFA
qsofa = pd.DataFrame(index=df.index)
qsofa['Resp'] = df['Resp'] >= 22
qsofa['SBP'] = df['SBP'] <= 100

df_qsofa = qsofa.sum(axis=1)

#Add feature qSOFA
df['qsofa'] = df_qsofa

In [ ]:
#SOFA
sofa = pd.DataFrame(index=df.index)
sofa['Platelets'] = 0
sofa['Platelets'][(400 <= df['Platelets']) & (df['Platelets'] < np.inf)] = 0
sofa['Platelets'][(300 <= df['Platelets']) & (df['Platelets'] < 400)] = 1
sofa['Platelets'][(200 <= df['Platelets']) & (df['Platelets'] < 300)] = 2
sofa['Platelets'][(100 <= df['Platelets']) & (df['Platelets'] < 200)] = 3
sofa['Platelets'][df['Platelets'] < 100] = 4

sofa['Bilirubin'] = 0
sofa['Bilirubin'][df['Bilirubin_total'] < 1.2] = 0
sofa['Bilirubin'][(1.2 <= df['Bilirubin_total']) & ((df['Bilirubin_total']) <= 1.9)] = 1
sofa['Bilirubin'][(1.9 < df['Bilirubin_total']) & ((df['Bilirubin_total']) <= 5.9)] = 2
sofa['Bilirubin'][(5.9 < df['Bilirubin_total']) & ((df['Bilirubin_total']) <= 11.9)] = 3
sofa['Bilirubin'][(11.9 < df['Bilirubin_total'])] = 4

sofa['Creatinine'] = 0
sofa['Creatinine'][df['Creatinine'] < 1.2] = 0
sofa['Creatinine'][(1.2 <= df['Creatinine']) & (df['Creatinine'] < 1.9)] = 1
sofa['Creatinine'][(1.9 <= df['Creatinine']) & (df['Creatinine'] < 3.5)] = 2
sofa['Creatinine'][(3.5 <= df['Creatinine']) & (df['Creatinine'] < 5)] = 3
sofa['Creatinine'][(df['Creatinine'] >= 5)] = 4

sofa['MAP'] = 0
sofa['MAP'][df['MAP'] < 70] = 1

df_sofa = sofa.sum(axis=1)

#Add feature SOFA
df['sofa'] = df_sofa

In [ ]:
#SIRS
df_sirs = pd.DataFrame(index=df.index, columns=['temp', 'hr', 'rr.paco2', 'wbc'])
df_sirs['temp'] = ((df['Temp'] > 38) | (df['Temp'] < 36))
df_sirs['hr'] = df['HR'] > 90
df_sirs['rr.paco2'] = ((df['Resp'] > 20) | (df['PaCO2'] < 32))
df_sirs['wbc'] = ((df['WBC'] < 4) | (df['WBC'] > 12))

# Sum each row, if >= 2 then mar as SIRS
df_sum = df_sirs.sum(axis=1) >= 2

# Add feature SIRS
df['SIRS'] = df_sum.astype(int) #save to 0 or 1

In [ ]:
#Add feature ShockIndex, BUN/CR
df['ShockIndex'] = df['HR']/df['SBP']
df['BUN/CR'] = df['BUN']/df['Creatinine']

In [ ]:
#MEWS
mews = pd.DataFrame(index=df.index)
mews['HR'] = 0
mews['HR'][df['HR']<30] = 3
mews['HR'][(30 <= df['HR']) & (df['HR'] < 40)] = 2
mews['HR'][(40 <= df['HR']) & (df['HR'] < 50)] = 1
mews['HR'][(50 <= df['HR']) & (df['HR'] < 100)] = 0
mews['HR'][(100 <= df['HR']) & (df['HR'] < 110)] = 1
mews['HR'][(110 <= df['HR']) & (df['HR'] <= 129)] = 2
mews['HR'][df['HR']>129] = 3

mews['SBP'] = 0
mews['SBP'][df['SBP']<70] = 3
mews['SBP'][(70 <= df['SBP']) & (df['SBP'] < 79)] = 2
mews['SBP'][(80 <= df['SBP']) & (df['SBP'] < 99)] = 1
mews['SBP'][(100 <= df['SBP']) & (df['SBP'] <= 199)] = 0
mews['SBP'][df['SBP']>199] = 2

mews['Temp'] = 0
mews['Temp'][df['Temp']<34] = 3
mews['Temp'][(34 <= df['Temp']) & (df['Temp'] < 34.9)] = 2
mews['Temp'][(35 <= df['Temp']) & (df['Temp'] < 35.9)] = 1
mews['Temp'][(36 <= df['Temp']) & (df['Temp'] < 37.9)] = 0
mews['Temp'][(38 <= df['Temp']) & (df['Temp'] <= 38.9)] = 1
mews['Temp'][df['Temp']>38.9] = 3

mews['Resp'] = 0
mews['Resp'][df['Resp']<9] = 2
mews['Resp'][(9 <= df['Resp']) & (df['Resp'] < 14)] = 0
mews['Resp'][(15 <= df['Resp']) & (df['Resp'] < 21)] = 1
mews['Resp'][(21 <= df['Resp']) & (df['Resp'] <= 29)] = 2
mews['Resp'][30 <= df['Resp']] = 3

df_mews = mews.sum(axis=1)

#Add feature MEWS
df['mews'] = df_mews

In [ ]:
#sofa_deterioration 
df['sofa_det'] = -1000
patients = list(df['patient'].unique())
for index, patient in enumerate(patients):
  df_patient = df[df['patient'] == patient] 
  df['sofa_det'][df['patient'] == patient] = (df_patient['sofa'] - df_patient['sofa'].shift(-24)>=2).astype(int)
  if (index % 2000 == 0):
    print(index)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0
2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000
24000
26000
28000
30000
32000
34000
36000
38000
40000


In [ ]:
df.to_pickle('data.pkl')